In [1]:
%cd ..

/Users/yehormishchyriak/Desktop/BonhamLab/summer2025/microbiome2function/dataset_curation


In [2]:
import pandas as pd
import scripts

In [3]:
# ENV
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
data = getenv("FETCHED_DATA")

In [5]:
df = pd.read_csv(data, index_col="Entry")

In [6]:
cleaned_df = scripts.clean_all_cols(df)

No preprocessing rule for 'Sequence' — leaving as is.


In [7]:
model_name = scripts.ESM2

In [8]:
model, tokenizer = scripts.get_AAseq_model_and_tokenizer(model_name)
# the warning below is not a problem because only the dense pooler is initialized at random, which doesn't concern
# the core transformer's weights. So, embeddings will still be accurate.

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
processed_df = scripts.process_ft_domain(cleaned_df.head(n=10), model, tokenizer) # testing

In [10]:
processed_df

,Domain [CC],Protein families,Gene Ontology (molecular function),Gene Ontology (biological process),Function [CC],Catalytic activity,EC number,Pathway,Rhea ID,Cofactor,domain_embedding
Entry,,,,,,,,,,,
P77499,NaN,"ABC transporter superfamily, Ycf16 family","(GO:0005524, GO:0016887)","(GO:0016226, GO:0009314)",has low atpase activity the sufbcd complex act...,NaN,NaN,NaN,NaN,NaN,"[0.06711539, -0.5962011, 0.4055503, 0.2615522,..."
P0DP70,NaN,Binding-protein-dependent transport system per...,NaN,"(GO:0015716, GO:0055085)",c-terminal fragment of the phne protein part o...,NaN,NaN,NaN,NaN,NaN,"[0.08803903, -0.5876103, 0.43695116, 0.2425640..."
Q8A8S8,NaN,LipB family,GO:0033819,GO:0036211,catalyzes the transfer of endogenously produce...,octanoyl-[ACP] + L-lysyl-[protein] = N(6)-octa...,2.3.1.181,protein modification protein lipoylation via e...,17665,NaN,"[0.07778977, -0.6027001, 0.4082584, 0.25359038..."
P76278,NaN,CopD family,NaN,GO:0006825,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P77378,NaN,ETF alpha-subunit/FixB family,"(GO:0009055, GO:0050660)",GO:0033539,may play a role in a redox process,NaN,NaN,NaN,NaN,NaN,NaN
A7ZLQ4,NaN,"4-oxalocrotonate tautomerase family, PptA subf...",GO:0016862,NaN,NaN,NaN,5.3.2.-,NaN,NaN,NaN,NaN
A0A2D1TVM1,NaN,"SEDS family, FtsW subfamily","(GO:0016757, GO:0015648)","(GO:0051301, GO:0071555, GO:0009252, GO:0008360)",peptidoglycan polymerase that is essential for...,[GlcNAc-(1->4)-Mur2Ac(oyl-L-Ala-gamma-D-Glu-L-...,2.4.99.28,cell wall biogenesis peptidoglycan biosynthesis,23708,NaN,NaN
A0A0J1ZC23,NaN,Xni family,"(GO:0008409, GO:0017108, GO:0003677, GO:000028...",GO:0033567,has flap endonuclease activity during dna repl...,NaN,3.1.-.-,NaN,NaN,"(K(+), Mg(2+))","[0.06719815, -0.5964499, 0.40536654, 0.2620830..."
W4UUZ6,NaN,"(TPP enzyme family, Acetolactate synthase smal...","(GO:0003984, GO:1990610, GO:0050660, GO:000028...","(GO:0009097, GO:0009099)",NaN,2 pyruvate + H(+) = (2S)-2-acetolactate + CO2,2.2.1.6,(amino-acid biosynthesis l-isoleucine biosynth...,25249,"(Mg(2+), thiamine diphosphate)","[0.10545829, -0.5911184, 0.4385609, 0.2311537,..."


In [11]:
embedding_example_esm = processed_df.loc["A0A0J1ZC23"].domain_embedding

In [12]:
txt_embedder = scripts.FreeTXTEmbedder()

In [21]:
non_nan_domain_ccs = cleaned_df["Domain [CC]"].notna()

In [22]:
string_example = cleaned_df[non_nan_domain_ccs].loc["Q8XBZ3"]["Domain [CC]"]

In [23]:
string_example

'domain i is involved in oligomerization and binding regulators domain ii is flexibile and of varying length in different bacteria domain iii forms the aaa region while domain iv binds dsdna'

In [57]:
embedding_example_openai = txt_embedder.request_embedding_for(string_example, scripts.SMALL_OPENAI_MODEL)

In [58]:
original_emb = embedding_example_openai[0]

In [59]:
made_up_similar_example = "domain i mediates oligomerization and regulator binding domain ii is flexible and varies in length among bacteria domain iii forms the aaa region domain iv binds dsdna"

In [60]:
made_up_different_example = "domain i binds calcium ions domain ii anchors in the cell membrane domain iii catalyzes phosphate transfer domain iv mediates protein complex formation"

In [61]:
fake_similar_emb = txt_embedder.request_embedding_for(made_up_similar_example, scripts.SMALL_OPENAI_MODEL)

In [62]:
fake_different_emb = txt_embedder.request_embedding_for(made_up_different_example, scripts.SMALL_OPENAI_MODEL)

In [63]:
fake_similar_emb = fake_similar_emb[0]; fake_different_emb = fake_different_emb[0]

In [64]:
import numpy as np

### L2:

In [65]:
dist_orig_sim = np.linalg.norm(original_emb - fake_similar_emb); dist_orig_diff = np.linalg.norm(original_emb - fake_different_emb)

In [66]:
dist_orig_sim

np.float64(0.29088045188771633)

In [67]:
dist_orig_diff

np.float64(0.8010285471668509)

### Cosine:

In [68]:
cos_orig_sim = np.dot(original_emb, fake_similar_emb) / (np.linalg.norm(original_emb) * np.linalg.norm(fake_similar_emb))

In [69]:
cos_orig_diff = np.dot(original_emb, fake_different_emb) / (np.linalg.norm(original_emb) * np.linalg.norm(fake_different_emb))

In [70]:
cos_orig_sim

np.float64(0.9576942857715264)

In [71]:
cos_orig_diff

np.float64(0.6791766265960767)